<a href="https://colab.research.google.com/github/GermanM3/GermanM3/blob/master/%EA%B3%B5%EB%A7%A4%EB%8F%84_%EC%9E%94%EA%B3%A0_%EB%88%84%EC%A0%81_%EA%B3%BC%EB%8B%A4(10%EC%9D%BC%EC%B9%98_%EA%B1%B0%EB%9E%98%EB%8C%80%EA%B8%88).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pykrx finance-datareader ta


In [28]:
from pykrx import stock
import FinanceDataReader as fdr
from datetime import datetime, timedelta
import pandas as pd

# 0. 날짜 설정
today = datetime.today()

# 공매도 데이터가 제공되는 가장 최근 영업일 찾기 (최대 7일 전까지 탐색)
# 실제 공매도 데이터는 T+2일 이후에 제공되므로, 적절한 날짜를 찾아야 합니다.
end_date_for_pykrx = None
for i in range(1, 8): # 최대 7일 전까지 탐색
    test_date = today - timedelta(days=i)
    try:
        # 삼성전자(005930)를 기준으로 공매도 데이터가 있는지 확인
        # get_shorting_status_by_date는 단일 종목 조회를 가정합니다.
        # 더 나은 방법은 get_shorting_balance_top50을 시도해 보는 것입니다.
        # 여기서는 get_shorting_balance_top50을 사용해 유효 날짜를 확인합니다.
        test_df = stock.get_shorting_balance_top50(test_date.strftime('%Y%m%d'))
        if not test_df.empty:
            end_date_for_pykrx = test_date.strftime('%Y%m%d')
            print(f"가장 최근 공매도 데이터 기준일: {end_date_for_pykrx}")
            break
    except Exception:
        continue

if not end_date_for_pykrx:
    print("가장 최근 공매도 데이터 기준일을 찾을 수 없습니다. 날짜를 수동으로 설정해주세요.")
    # 예시: end_date_for_pykrx = "20250521"
    exit()

# FinanceDataReader용 날짜 형식
end_date_fdr = datetime.strptime(end_date_for_pykrx, "%Y%m%d")
# 10 영업일 평균 거래대금을 위해 넉넉히 20일 전부터 데이터 요청
start_date_fdr = end_date_fdr - timedelta(days=20)


# 1. KOSPI 200 티커 리스트 받기 (참고용. 실제 분석 대상은 공매도 상위 50 종목)
# KOSPI 200 지수 코드 "1028"을 사용합니다.
kospi200_tickers_list = stock.get_index_portfolio_deposit_file("1028")
# set으로 변환하여 검색 속도 향상
kospi200_tickers_set = set(kospi200_tickers_list)
print(f"KOSPI 200 종목 수: {len(kospi200_tickers_list)}")


# 2. 공매도 잔고 상위 50 종목 정보 가져오기 (시간 단축 핵심!)
# 이 함수는 공매도 잔고 수량('공매도잔고')을 직접 제공합니다.
df_short_balance_top50 = stock.get_shorting_balance_top50(date=end_date_for_pykrx, market="KOSPI")
# 인덱스가 티커이므로 바로 사용 가능
print(f"공매도 잔고 상위 50개 종목 수집 완료: {len(df_short_balance_top50)} 종목")


# 3. KOSPI 200이면서 공매도 잔고 상위 50인 종목만 필터링
# 공매도 잔고 상위 50 종목 중 KOSPI 200에 속하는 종목만 선택
target_tickers = [
    ticker for ticker in df_short_balance_top50.index
    if ticker in kospi200_tickers_set
]
# 필터링된 공매도 잔고 데이터프레임
df_short_balance_filtered = df_short_balance_top50.loc[target_tickers].copy()
print(f"KOSPI 200 내 공매도 잔고 상위 종목 (최종 분석 대상): {len(df_short_balance_filtered)} 종목")


# 4. 필터링된 종목들에 대해 최근 10일 거래대금 평균 구하기 (FDR 활용)
# 이제 소수의 종목에 대해서만 FDR을 호출하므로 훨씬 빠릅니다.
avg_trade_value_dict = {}

for ticker in df_short_balance_filtered.index:
    try:
        df_fdr = fdr.DataReader(ticker, start=start_date_fdr.strftime("%Y-%m-%d"), end=end_date_fdr.strftime("%Y-%m-%d"))
        if not df_fdr.empty:
            df_fdr['거래대금'] = df_fdr['Close'] * df_fdr['Volume']
            # 최근 10 영업일 거래대금 평균 (tail(10)은 영업일 기준 10개)
            avg_value = df_fdr['거래대금'].tail(10).mean()
            avg_trade_value_dict[ticker] = avg_value
        else:
            avg_trade_value_dict[ticker] = None # 데이터가 없으면 None
    except Exception as e:
        # print(f"FDR 오류 - {ticker}: {e}") # 디버깅용
        avg_trade_value_dict[ticker] = None # 오류 발생 시 None

# 5. 공매도 잔고 DataFrame에 10일평균거래대금 컬럼 추가
df_short_balance_filtered['10일평균거래대금'] = df_short_balance_filtered.index.map(lambda x: avg_trade_value_dict.get(x, None))

# 거래대금 데이터가 없는 종목은 제거 (None 값 제거)
df_short_balance_filtered.dropna(subset=['10일평균거래대금'], inplace=True)
print(f"10일 평균 거래대금 데이터 추가 완료. 남은 종목 수: {len(df_short_balance_filtered)}")



가장 최근 공매도 데이터 기준일: 20250520
KOSPI 200 종목 수: 200
공매도 잔고 상위 50개 종목 수집 완료: 50 종목
KOSPI 200 내 공매도 잔고 상위 종목 (최종 분석 대상): 26 종목
10일 평균 거래대금 데이터 추가 완료. 남은 종목 수: 26

조건을 만족하는 종목이 없습니다.


In [44]:
# 6. 공매도 금액 / 10일평균거래대금 계산
# 분모가 0이 되는 경우를 방지합니다.
df_short_balance_filtered['금액_대_10일거래대금비율'] = df_short_balance_filtered.apply(
    lambda row: row['공매도금액'] / row['10일평균거래대금'] if row['10일평균거래대금'] != 0 else float('inf'), axis=1
)

# 7. 기준 10 이상인 종목 필터링
df_final_filtered = df_short_balance_filtered[df_short_balance_filtered['금액_대_10일거래대금비율'] >= 10]

# 결과 출력
if not df_final_filtered.empty:
    # 종목명 추가 (출력을 위해)
    df_final_filtered['종목명'] = df_final_filtered.index.map(stock.get_market_ticker_name)
    print("\n조건 (금액_대_10일거래대금비율 >= 10)을 만족하는 종목:")
    print(df_final_filtered.sort_values('금액_대_10일거래대금비율', ascending=False))
else:
    print("\n조건 (금액_대_10일거래대금비율 >= 10)을 만족하는 종목이 없습니다.")
    # 조건 만족 종목이 없을 경우, 전체 대상 종목 중 비율 높은 순서대로 상위 5개 출력
    if not df_short_balance_filtered.empty: # 필터링 전 데이터가 있다면
        print("대신, '금액_대_10일거래대금비율'이 높은 상위 5개 종목을 출력합니다:")
        # 비율을 기준으로 내림차순 정렬 후 상위 5개 선택
        top_5_by_ratio = df_short_balance_filtered.sort_values('금액_대_10일거래대금비율', ascending=False).head(5)
        # 종목명 추가 (출력을 위해)
        top_5_by_ratio['종목명'] = top_5_by_ratio.index.map(stock.get_market_ticker_name)
        print(top_5_by_ratio)
    else:
        print("분석 대상 종목 데이터가 없습니다.")


조건 (금액_대_10일거래대금비율 >= 10)을 만족하는 종목이 없습니다.
대신, '금액_대_10일거래대금비율'이 높은 상위 5개 종목을 출력합니다:
        순위    공매도잔고      상장주식수         공매도금액          시가총액        비중  \
티커                                                                     
361610  28  1064126   71297592   22399852300  1.500814e+12  1.490234   
373220  42  2774911  234000000  775587624500  6.540300e+13  1.190430   
096770  21  2431718  151034776  205480171000  1.276244e+13  1.610352   
010950  50  1186975  112582792   60179632500  5.707948e+12  1.049805   
005420  10   819978   38408228   11200899480  5.246564e+11  2.130859   

           10일평균거래대금  잔고_대_10일거래대금비율  금액_대_10일거래대금비율         종목명  
티커                                                                
361610  2.558639e+09        0.000416        8.754598  SK아이이테크놀로지  
373220  9.158824e+10        0.000030        8.468201    LG에너지솔루션  
096770  2.526081e+10        0.000096        8.134346     SK이노베이션  
010950  7.461234e+09        0.000159        8.065641       S-Oil  
005420  

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
